## SECTION A                                                                                                                        

In [1]:
#import sys
#!{sys.executable} -m pip install omdb
import omdb
import json
from ipywidgets import widgets           # widgets
from IPython.display import clear_output # Clear function - to give terminal a cleaner look during lookup
omdb.set_default('apikey', '2c22860f')   # default api key setting
  
movie_details = []  #array for temporary movie information

def start():                                    #Main Menu 
    clear_output()
    print(color.BOLD +"WELCOME! This Interface will enable you to search for movies on OMDb\n"+color.END) 
    if len(open('movie_file.txt','r+').read())>1:      #check if data in file to read
        with open('movie_file.txt','r+') as file:                         #'with open' best practice. automatically closes file
            print(color.DARKCYAN+"The following movies are already on file:\n"+color.END)
            data = json.load(file)  
            x=0
            for p in data:                    # show movies previously searched
                x+=1
                print('Title: '+p['title'], ', '+p['release'][7:])
    print(color.RED +"\nPlease select the number that corresponds with your selection within the menus."+color.END)
            
    selection = int(input("Press 1 to search for a movie\nPress 2 to access previously searched data \nPress 3 to exit\n"))
    while selection <1 or selection>3:                       #loop to verify numeric input
        clear_output()
        print(color.RED+"Invalid numeric selection. "+color.END+"Try again.")
        selection = int(input("\nPress 1 to search for a new movie\nPress 2 to access previously searched data \nPress 3 to exit\n"))
    
    if selection==1:
        sub_menu()
    elif selection==2:
        clear_output()
        file_menu()
    elif selection==3:
        clear_output()
        print('\nGoodbye!')
        exit()            

def sub_menu():  # Search for movies in this menu 
    clear_output()
    selection=int(input(color.BOLD +"Search Menu:"+color.END +"\n\nPress 1 to search by title/keyword\nPress 2 to search by title and year\nPress 3 to search by IMDB ID\n\n"))
    while selection<1 or selection>3: #loop to verify numeric input
        print(color.RED+"Invalid selection. "+color.END+"Try again.")
        selection=int(input("Search Menu:\n\nPress 1 to search by title/keyword\nPress 2 to search by title and year\nPress 3 to search by IMDB ID\n\n"))
    
    if selection==1:                            # Search by keyword/ title only - basic search
        clear_output()
        keyword_search = input("Please enter the movie title: ")
        movie = omdb.get(title=keyword_search, fullplot=True, media_type='movie')   #retrieve movie info from OMDb
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            keyword_search = input("Please enter the movie title: ")
            movie = omdb.get(title=keyword_search, fullplot=True)
        clear_output()     
        table(movie)
        if len(open('movie_file.txt','r+').read())>1:
            create_dict(movie)                  # Creates new dictionary to write in list
        else:
            create_initial_dict(movie)         # On first run
        do_now()
            
    elif selection==2:                         # Search by title and year - advanced search
        clear_output()
        keyword_search = input("Please enter the movie title: ")
        year_search = input("Please enter the movie release year: ")
        movie = omdb.get(title=keyword_search,year=year_search, fullplot=True,media_type='movie') #retrieve movie info from OMDb
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            keyword_search = input("Please enter the movie title: ")
            year_search = input("Please enter the movie release year: ")
            movie = omdb.get(title=keyword_search,year=year_search, fullplot=True)

        clear_output()
        table(movie)       #send movie info to table widget for cleaner display of information
        if len(open('movie_file.txt','r+').read())>1:
            create_dict(movie)
        else:
            create_initial_dict(movie)
        do_now()

    elif selection==3:                         # Search by IMDB ID
        clear_output()
        imdb_search = input("Please enter the IMDB ID: ")   # find movie with it's IMDB ID
        movie = omdb.get(imdbid=imdb_search, fullplot=True,media_type='movie')
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            imdb_search = input("Please enter the IMDB ID: ")
            movie = omdb.get(imdbid=imdb_search, fullplot=True)
            
        clear_output()   
        table(movie)
        if len(open('movie_file.txt','r+').read())>1:    # determining which function to be called
            create_dict(movie)                            
        else:
            create_initial_dict(movie)
        do_now()
        
def do_now():           # Options displayed to users after movie has been searched
    selection = int(input(color.BOLD+"What would you like to do now?\n\n"+color.END+"Press 1 to search again\nPress 2 for main menu\nPress 3 to exit\n"))
    while selection !=1 and selection!=2 and selection!=3:                       #loop to verify numeric input
        clear_output()
        print(color.RED+"Invlaid numeric selection. Try again."+color.END)
        selection = int(input("\nPress 1 to search again\nPress 2 for main menu\nPress 3 to exit\n"))
        
    if selection==1:
        sub_menu()
    elif selection==2:
        start()     
    elif selection==3:
        print('\nGoodbye!')
        exit()
              
def file_menu():         # For accessinf the information on the file (list of movies)
    clear_output()
    i=1
    print(color.UNDERLINE+"Welcome to the file menu. From here you can access archived search data.\n"+color.END)

    if len(open('movie_file.txt','r+').read())<1:      #check for data in file to read
        clear_output()
        print("There are no archived movies yet. Press 'B' to go back to the main menu.")
        check = input()
        while check.upper() != 'B':
            print(color.RED+"\nInvalid entry."+color.END+" Try again.\n")
            check = input()
        start()
        
    print(color.DARKCYAN+color.BOLD+"Movies already on file:\n"+color.END)    
    with open('movie_file.txt','r+') as file:
        data = json.load(file)
        meta = 0
        x=len(data)            #for assigning numerical value to list data
        y=len(data)            # for metascore calculation purposes
        while i<x+1:
            for p in data:
                print(str(i)+'.','Title: '+ p['title'])
                try: 
                    meta = (meta + int(p['metascore']))
                except ValueError:
                    print("   No Metascore available. "+p['title']+" excluded from calculation.")
                    y-=1    # decrease amount of movies by one to calculate correct average
                    pass
                i+=1
            try:
                print(color.UNDERLINE+'\nMean Metascore:'+color.END,round((meta/y),2),'%')
            except ZeroDivisionError:  # when the only movie on file does not have a metascore, the 'y' variable will be zero
                pass
    print("\nEnter the"+color.BOLD+" number"+color.END+" of the movie you would like more information on OR\n\nEnter 'B' to go back\nEnter 'S' to save file and exit\nEnter 'X' to clear file and go back\n")
    sel = input()
    while sel.upper() != 'B' and sel.upper() != 'S' and sel.upper() != 'X':
        try:
            if int(sel)>0 and int(sel)<x+1:   #convert the string input to integer
                break               # This accepts the integer values assigned to the movies in the file

        except ValueError:
            pass                     #allows integers or strings to be passed as input after they have been invalidated.
        
        clear_output()
        print(color.RED+"Invalid entry."+color.END+" Try again.\n")
        sel = input(print("Enter the"+color.BOLD+" number"+color.END+" of the movie you would like more information on OR\n\nEnter 'B' to go back\nEnter 'S' to save file and exit\nEnter 'X' to clear file and go back\n"))
    
    if sel.upper() != 'B' and sel.upper() != 'S' and sel.upper() != 'X':
        if int(sel)>0 and int(sel)<x+1:
            with open('movie_file.txt','r') as file2:            # displaying movie information in file menu 
                clear_output()
                data = json.load(file2) 
                entry= data[int(sel)-1]                         # Entire dictionary is loaded from list stored in file
                print(color.BOLD+"Movie Information: \n"+color.END)
                print('Title: ' + entry['title'])
                print('Genre: ' + entry['genre'])
                print('Released: ' + entry['release'])
                print('Director: ' + entry['director'])
                print('Actor(s): ' + entry['actors'])
                print('Runtime: ' + entry['runtime'])
                print('Metascore: ' + entry['metascore'])
                print('Rated: ' + entry['rated'])
                print('Awards: ' + entry['awards'])
            
                selection= input("Press 'B' to go back.")
                while selection.upper() != 'B':
                    print(color.RED+"\nInvalid entry."+color.END+" Press 'B' to go back.\n")
                    selection = input()
                if selection.upper()=='B':
                    file_menu()    
    
    if sel.upper()=='B':
        start()     
    elif sel.upper()=='X':
        clear()
        start()
    elif sel.upper()=='S':
        clear_output()
        print("Goodbye!")
        exit()

def create_dict(movie):                  # For adding dictionaries to the text file   
    dictionary = ({
            'title' : movie.get('title',0),
            'genre' : movie.get('genre',0),
            'release' : movie.get('released',0),
            'director' : movie.get('director',0),
            'actors': movie.get('actors',0),
            'plot'  : movie.get('plot',0),
            'rated' : movie.get('rated',0),
            'awards': movie.get('awards',0),
            'metascore' : movie.get('metascore',0),
            'runtime' : movie.get('runtime',0)
        })
    
    with open ('movie_file.txt', mode='r+') as file:       # r+ for reading and writing
        file.seek(0,2)
        position = file.tell() -1                          # determining the last character position in the text file
        file.seek(position)
        file.write( ",{}]".format(json.dumps(dictionary)) )
        
def create_initial_dict(movie):             # creates initial entry in list in text file                 
    movie_details.append({
            'title' : movie.get('title',0),
            'genre' : movie.get('genre',0),
            'release' : movie.get('released',0),
            'director' : movie.get('director',0),
            'actors': movie.get('actors',0),
            'plot'  : movie.get('plot',0),
            'rated' : movie.get('rated',0),
            'metascore' : movie.get('metascore',0),
            'awards': movie.get('awards',0),
            'runtime' : movie.get('runtime',0)
        })
    with open('movie_file.txt', 'r+') as file: 
        json.dump(movie_details, file)              # dumps - json version for writing to a file
    
def clear():                                       #clears output
    with open('movie_file.txt','w') as f:
        f.truncate(0)   
    movie_details = []                            # clears array
    
def table(movie):   # Display table widget
    print(color.GREEN+color.BOLD+"\nMovie Found!"+color.END+" See the below table for details!")
    m_t = widgets.HTML(value=movie.get('title',0))
    m_g = widgets.HTML(value=movie.get('genre',0))
    m_y = widgets.HTML(value=movie.get('released',0))
    m_d = widgets.HTML(value=movie.get('director',0))
    m_c = widgets.HTML(value=movie.get('actors',0))
    m_p = widgets.HTML(value=movie.get('plot',0))
    m_r = widgets.HTML(value=movie.get('rated',0))
    m_a = widgets.HTML(value=movie.get('awards',0))
    m_x = widgets.HTML(value=movie.get('runtime',0))

    tab_nest = widgets.Tab()
    tab_nest.children = [m_t,m_g,m_y,m_d,m_c,m_p,m_r,m_a,m_x]
    tab_nest.set_title(0, 'Title')
    tab_nest.set_title(1, 'Genre')
    tab_nest.set_title(2, 'Released')
    tab_nest.set_title(3, 'Director')
    tab_nest.set_title(4, 'Actors')
    tab_nest.set_title(5, 'Plot')
    tab_nest.set_title(6, 'Rated')
    tab_nest.set_title(7, 'Awards')
    tab_nest.set_title(8, 'Runtime')  
    display(tab_nest)
           
class color:                                     # for editing text
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    UNDERLINE = '\033[4m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    END = '\033[0m'

start()       


Goodbye!


# SECTION B: 

In [ ]:
import sys
#!{sys.executable} -m pip install omdb   #installing the pip
import omdb
import json                              #json format
from ipywidgets import widgets           # widgets
from IPython.display import clear_output # Clear function - to give terminal a cleaner look during lookup
omdb.set_default('apikey', '2c22860f')   # default api key setting
  
movie_details = []  #array for temporary movie information

In the following section the thought process behind the coding of the program will be discussed. While the prgram is working as specified in the objective, there were many obstacles and challenges that had to be overcome.

These are a few requirements I set out for myself with the coding of this program:
- I wanted a robust program that is fault tolerant. The user must not be able to crash the program.
- I wanted the program to make use of some GUI elements without complicating the code too much.
- I wanted to meet the minimum specifications as set out by the objective.
- I wanted to add extra features that are usable and worth implementing.
- I wanted my menus to be distinguishable and easy to navigate.
- I did not want complicated code.

The code above was mostly imports. The most important one of course was the 'import omdb'. This allowed the program to gain access to the unique OMDB functions and calls used to interact with the API. The json import allowed me write to the text file using the json format. This allowed me to keep the dictionaries containing movie information and load them again when the program requests it. 

The widget import was for displaying buttons/tables. While I decided not to use a full GUI, I still wanted to display the movie information in a structured and clean way. The table widget allowed me this feature so when the user searched for a movie, they would see a table which displayed information in a way that is easy to understand. The 'clear_ouput' import was for when I wanted to clear the terminal of any information. This was purely for keeping the console clean and focused.

The 'omdb.set_default' allowed me to declare my API key once so that I would not have to enter it repeatedly within the prgram when making requests to the OMDB API.



In [ ]:
def start():                                    #Main Menu 
    clear_output()
    print(color.BOLD +"WELCOME! This Interface will enable you to search for movies on OMDb\n"+color.END) 
    if len(open('movie_file.txt','r+').read())>1:      #check if data in file to read
        with open('movie_file.txt','r+') as file:                         #'with open' best practice. automatically closes file
            print(color.DARKCYAN+"The following movies are already on file:\n"+color.END)
            data = json.load(file)  
            x=0
            for p in data:                    # show movies previously searched
                x+=1
                print('Title: '+p['title'], ', '+p['release'][7:])
    print(color.RED +"\nPlease select the number that corresponds with your selection within the menus."+color.END)
            
    selection = int(input("Press 1 to search for a movie\nPress 2 to access previously searched data \nPress 3 to exit\n"))
    while selection <1 or selection>3:                       #loop to verify numeric input
        clear_output()
        print(color.RED+"Invalid numeric selection. "+color.END+"Try again.")
        selection = int(input("\nPress 1 to search for a new movie\nPress 2 to access previously searched data \nPress 3 to exit\n"))
    
    if selection==1:
        sub_menu()
    elif selection==2:
        clear_output()
        file_menu()
    elif selection==3:
        clear_output()
        print('\nGoodbye!')
        exit()         


The code above displays the menu the user sees first. It is a defined function which is called on execution and then throughout the program whenever the user needs to access the main menu. The challenge for this part of the program was getting two versions of the menu to display once called; these versions included a main menu where the menu had to display previously searched movie titles and another where there were no movies to display.I solved this by simply determining if there was any text written to the file using the len() method. 

If there are movies on the file, is is loaded using the json format and the movie titles are displayed individually using a for loop. Using json allowed me to load the dictionaries from the file, as they were written without any conversion to the html or xml formats. This was also helpful as OMDB requests are generally formatted as JSON (PyPi, 2018). 

Within this menu the user is able to to navigate to two other menus or exit the program. The input the user provides is also checked for any value errors. The first option takes the user to the search sub menu where they will be able to search for movies using different searching criteria. The second option will allow the user to access the file menu where they will be able to access and load information that is currently written on the file. The third option terminates the program.

In [ ]:
def sub_menu():  # Search for movies in this menu 
    clear_output()
    selection=int(input(color.BOLD +"Search Menu:"+color.END +"\n\nPress 1 to search by title/keyword\nPress 2 to search by title and year\nPress 3 to search by IMDB ID\n\n"))
    while selection<1 or selection>3: #loop to verify numeric input
        print(color.RED+"Invalid selection. "+color.END+"Try again.")
        selection=int(input("Search Menu:\n\nPress 1 to search by title/keyword\nPress 2 to search by title and year\nPress 3 to search by IMDB ID\n\n"))
    
    if selection==1:                            # Search by keyword/ title only - basic search
        clear_output()
        keyword_search = input("Please enter the movie title: ")
        movie = omdb.get(title=keyword_search, fullplot=True, media_type='movie')   #retrieve movie info from OMDb
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            keyword_search = input("Please enter the movie title: ")
            movie = omdb.get(title=keyword_search, fullplot=True)
        clear_output()     
        table(movie)
        if len(open('movie_file.txt','r+').read())>1:
            create_dict(movie)                  # Creates new dictionary to write in list
        else:
            create_initial_dict(movie)         # On first run
        do_now()

The sub_menu() function is called when the user enters the first option of three to search for a movie in the main menu. The sub menu provides the user with the options of searching for a movie using three different criteria.

- *Criteria 1:* Only providing the program with the movie title or start of the movie title (keyword).
- *Criteria 2:* Provide the program with the movie's title and the year it was released in.
- *Criteria 3:* Provide the program with an IMDB ID.

If the input provided by the user does not meet the criteria, they are asked to try again. The above code focuses on only criteria 1. this is the 1st instance where a call is made to the omdb API. *The OMDB API is a restful API which uses HTTP requests to get data.* The HTTP requests can also post or delete data but for this program we only needed to use the GET method. A REST (Representational State Transfer) API generally provides a lot of flexibility through being able to use existing protocols without needing to install additional software (Mulesoft, 2019). 

To search for a movie title I used the omdb.get() method which allowed me to search for a title with the 'title' parameter. The .get() method is considered to be a generic request which accepts keywords. This suited me as the user will not always remember the exact title of the movie they are looking for. In my .get() method I also included the fullplot parameter as I wanted to display the plot of the movie within the table and save it to the file if necessary.

To determine whether a lookup was successful, I measured the length of the data within the returned 'movie' data. If it returned a value larger than 0, I knew that the search was successful. The table() method displays the 'movie' data in a table that is easy to read and navigate through. The table provides some basic and extra information on the movie that was searched. It is important for me to code the program in the order that I think the user will most likely use it. The order is also important for displaying information in the correct order. The following two methods, create_dict() and create_initial_dict() was my solution to a problem I struggled with. This will be discussed later but for now it is important to know that I had to create an initial dictionary entry (when the file was empty and it is the first lookup) and dictionaries that followed the first one.

do_now() is a function that displays a menu which allows the user navigation options. 

*Reflection: I should have fleshed out the program and made sure that all the menus interact and correspond with each other correctly before adding extra features. This complicated the code and made it difficult to find errors. When I rewrote the code I made sure to code the basic structure first before adding extra features.*


In [ ]:
elif selection==2:                         # Search by title and year - advanced search
        clear_output()
        keyword_search = input("Please enter the movie title: ")
        year_search = input("Please enter the movie release year: ")
        movie = omdb.get(title=keyword_search,year=year_search, fullplot=True,media_type='movie') #retrieve movie info from OMDb
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            keyword_search = input("Please enter the movie title: ")
            year_search = input("Please enter the movie release year: ")
            movie = omdb.get(title=keyword_search,year=year_search, fullplot=True)

        clear_output()
        table(movie)       #send movie info to table widget for cleaner display of information
        if len(open('movie_file.txt','r+').read())>1:
            create_dict(movie)
        else:
            create_initial_dict(movie)
        do_now()

    elif selection==3:                         # Search by IMDB ID
        clear_output()
        imdb_search = input("Please enter the IMDB ID: ")   # find movie with it's IMDB ID
        movie = omdb.get(imdbid=imdb_search, fullplot=True,media_type='movie')
        while(len(movie)<1):                       # validate input
            print('\nNo movie found. Try again.')
            imdb_search = input("Please enter the IMDB ID: ")
            movie = omdb.get(imdbid=imdb_search, fullplot=True)
            
        clear_output()   
        table(movie)
        if len(open('movie_file.txt','r+').read())>1:    # determining which function to be called
            create_dict(movie)                            
        else:
            create_initial_dict(movie)
        do_now()

The above code is more or less a copy of the process behind the first selection. It shows how the movie information for option 2 and option 3 is  called using different OMDB parameters namely 'year' and 'imdbid'. These parameters allows the user to refine their search. On every successful request, selected movie information is added into a dictionary which is then added to a list which is written to a file in the creat_dict() or create_initial_dict() methods.

In [ ]:
def do_now():           # Options displayed to users after movie has been searched successfully
    selection = int(input(color.BOLD+"What would you like to do now?\n\n"+color.END+"Press 1 to search again\nPress 2 for main menu\nPress 3 to exit\n"))
    while selection !=1 and selection!=2 and selection!=3:                       #loop to verify numeric input
        clear_output()
        print(color.RED+"Invlaid numeric selection. Try again."+color.END)
        selection = int(input("\nPress 1 to search again\nPress 2 for main menu\nPress 3 to exit\n"))
        
    if selection==1:
        sub_menu()
    elif selection==2:
        start()     
    elif selection==3:
        print('\nGoodbye!')
        exit()

The do_now() function is called after every successful search of a movie in the sub_menu() method. The do_now() method is used to provide the user with the option of searching for a movie again or going back to the main menu. The input is also verified using a while loop .

In [ ]:
def file_menu():         # For accessinf the information on the file (list of movies)
    clear_output()
    i=1
    print(color.UNDERLINE+"Welcome to the file menu. From here you can access archived search data.\n"+color.END)
    if len(open('movie_file.txt','r+').read())<1:      #check for data in file to read
        clear_output()
        print("There are no archived movies yet. Press 'B' to go back to the main menu.")
        check = input()
        while check.upper() != 'B':
            print(color.RED+"\nInvalid entry."+color.END+" Try again.\n")
            check = input()
        start()
        
    print(color.DARKCYAN+color.BOLD+"Movies already on file:\n"+color.END)    
    with open('movie_file.txt','r+') as file:
        data = json.load(file)
        meta = 0
        x=len(data)            #for assigning numerical value to list data
        y=len(data)            # for metascore calculation purposes
        while i<x+1:
            for p in data:
                print(str(i)+'.','Title: '+ p['title'])
                try: 
                    meta = (meta + int(p['metascore']))
                except ValueError:
                    print("   No Metascore available. "+p['title']+" excluded from calculation.")
                    y-=1    # decrease amount of movies by one to calculate correct average
                    pass
                i+=1
            try:
                print(color.UNDERLINE+'\nMean Metascore:'+color.END,round((meta/y),2),'%')
            except ZeroDivisionError:     # when the only movie on file does not have a metascore, the 'y' variable will be zero
                pass
    print("\nEnter the"+color.BOLD+" number"+color.END+" of the movie you would like more information on OR\n\nEnter 'B' to go back\nEnter 'S' to save file and exit\nEnter 'X' to clear file and go back\n")
    sel = input()
    while sel.upper() != 'B' and sel.upper() != 'S' and sel.upper() != 'X':
        try:
            if int(sel)>0 and int(sel)<x+1:   #convert the string input to integer
                break               # This accepts the integer values assigned to the movies in the file

        except ValueError:
            pass                     #allows integers or strings to be passed as input after they have been invalidated.
        
        clear_output()
        print(color.RED+"Invalid entry."+color.END+" Try again.\n")
        sel = input(print("Enter the"+color.BOLD+" number"+color.END+" of the movie you would like more information on OR\n\nEnter 'B' to go back\nEnter 'S' to save file and exit\nEnter 'X' to clear file and go back\n"))
    
    if sel.upper() != 'B' and sel.upper() != 'S' and sel.upper() != 'X':
        if int(sel)>0 and int(sel)<x+1:
            with open('movie_file.txt','r') as file2:            # displaying movie information in file menu 
                clear_output()
                data = json.load(file2) 
                entry= data[int(sel)-1]                         # Entire dictionary is loaded from list stored in file
                print(color.BOLD+"Movie Information: \n"+color.END)
                print('Title: ' + entry['title'])
                print('Genre: ' + entry['genre'])
                print('Released: ' + entry['release'])
                print('Director: ' + entry['director'])
                print('Actor(s): ' + entry['actors'])
                print('Runtime: ' + entry['runtime'])
                print('Metascore: ' + entry['metascore'])
                print('Rated: ' + entry['rated'])
                print('Awards: ' + entry['awards'])
            
                selection= input("Press 'B' to go back.")
                while selection.upper() != 'B':
                    print(color.RED+"\nInvalid entry."+color.END+" Try again.\n")
                    selection = input()
                if selection.upper()=='B':
                    file_menu()      
    if sel.upper()=='B':
        start()     
    elif sel.upper()=='X':
        clear()
        start()
    elif sel.upper()=='S':
        clear_output()
        print("Goodbye!")
        exit()

The file_menu() function was the most challenging part of this program. I went through multiple iterations of this function to test how different formats reacted when read from the text file. I quickly learnt that reading multiple dictionaries from a text file and displaying them according to a numeric value was not easy. Reading a list containing dictionaries from a file with the .read() method was not a viable method for what I wanted to achieve. 

The function begins with testing if there is any data written on the text file. If not, it prompts the user to press 'B' to go back to the main menu as there is no data to work with. If there is data on the file, it is assigned onto the 'data' variable to be worked with. As I am requesting the movie information in JSON and dumping it as such into a file, it is important that I also load the information from the file using the json.load() method. This means that there was no conversion of the data between it being dumped to the file and it being loaded to a variable to be displayed. An additional feature I implemented was calculating the average metascore of all movies on the file. This was done using the for loop that read out the movie titles. The program will also determine whether the movie has a metascore and if none is given the movie is excluded from the calculation.

The file menu allows the user to view information on existing movies on the file, to clear the file, or to save the file as is and exit the program. Movies are accessed by entering the numeric value assigned to them with the for loop. To interact with the file and menus the user can enter a select few numbers and strings such as 'X', 'B' and 'S'. In order to validate the input from the user, as they are allowed to enter both strings and integers, I had to make use of an value exception block. This means the user can only enter the values shown to them on the menu. 

To display a chosen movie's details, the user must enter the option by selecting the movie's corresponding number. The number is generated by taking the length of the 'data' variable, which is the amount of dictionaries in a list within the file.
I used a for and while loop to do this. 
After the movie is selected, its details are shown with the option of pressing 'B' to go back. As data entries within a list start with '0', I had to increment the file postion by one when the user selects the movie and then decrease it by one when calling its location within the list. With this I avoid a situation where the user had to pick a movie numbered 0. 

Using the json.load() method I can load in the list from the text file and select the dictionary containing the movie details I want to display using the number provided by the user. This number remains the number of that specific movie until the user clears the file. 

*Reflection: I struggled with the different formats. Initially I requested all movie data in xml format from the OMDB API but quickly learnt that when reading in lists or dictionaries from the file, I was only reading in strings and not the dictionaries containing the string elements. This problem was solved using JSON.*

In [ ]:
def create_dict(movie):                  # For adding dictionaries to the text file   
    dictionary = ({
            'title' : movie.get('title',0),
            'genre' : movie.get('genre',0),
            'release' : movie.get('released',0),
            'director' : movie.get('director',0),
            'actors': movie.get('actors',0),
            'plot'  : movie.get('plot',0),
            'rated' : movie.get('rated',0),
            'awards': movie.get('awards',0),
            'metascore' : movie.get('metascore',0),
            'runtime' : movie.get('runtime',0)
        })
    
    with open ('movie_file.txt', mode='r+') as file:       # r+ for reading and writing
        file.seek(0,2)
        position = file.tell() -1                          # determining the last character position in the text file
        file.seek(position)
        file.write( ",{}]".format(json.dumps(dictionary)) )
        
def create_initial_dict(movie):             # creates initial entry in list in text file                 
    movie_details.append({
            'title' : movie.get('title',0),
            'genre' : movie.get('genre',0),
            'release' : movie.get('released',0),
            'director' : movie.get('director',0),
            'actors': movie.get('actors',0),
            'plot'  : movie.get('plot',0),
            'rated' : movie.get('rated',0),
            'metascore' : movie.get('metascore',0),
            'awards': movie.get('awards',0),
            'runtime' : movie.get('runtime',0)
        })
    with open('movie_file.txt', 'r+') as file: 
        json.dump(movie_details, file)              # dumps - json version for writing to a file

The above two functions perform the same task which is to write data to the 'movie_file.txt' file. However, the first function is only called if there is already data written on the file. The second function, create_initial_dict(), is only called when it is the first movie that is being written onto the text file. Using two seperate functions was my choice, although it was probably not necessary. 

Both functions receive the movie variable as an argument, it then uses the 'movie' variable, which is a JSON dictionary containing the movie information requested from OMDB, to create a new dictionary with fewer details. The data pulled from the 'movie' variable can be changed depending on what information is needed. With the 'create_intitial_dict()' function, this first dictionary is appended to the 'movie_details' array and *dumped* to the text file. This does two things: 
1. It creates a list of dictionaries within the text file. This list can now be appended with other dictionaries. 
2. It allows us to assign a positional value for every movie that has been searched within the text file.
Both of these characteristics enables us to better sort and load our movie information.

It can also be noted that we open our text file in the 'r+'-mode so that we can both read and write to it when opening it. Opening in 'w'-mode will clear our text file.

create_dict() however does not just append dictionaries containing movie information in the traditional sense. For it to add dictionaries to an already defined list in the text file, it has to manually remove the last character in the list (']'). When the last character is removed, thereby opening the list, dumping a new dictionary into the existing list is made possible. The program then has to close the list again. Using 'file.seek()' we can determine where the last character in our text file is located. This way of adding dictionaries is necessary for the program to work.

*Reflection: This was a difficult feature to add. I had to scavenge github and stackoverflow for any information related to reading and writing to an already defined dictionary within a file. It almost seemed impossible. I had coded to the point where I could only run the program when the text file was empty, the initial list containing the movie dictionaries would be successfully written to the file but the program could not load it again once another dictionary was added to the file. This was, as I later realised, due to the fact that the program was dumping the new dictionaries after the initially defined list, and not **in** it. The code above solves that problem by opening the list and then dumping the new dictionaries containing movie information.*


In [ ]:
def table(movie):   # Display table widget
    print(color.GREEN+color.BOLD+"\nMovie Found!"+color.END+" See the below table for details!")
    m_t = widgets.HTML(value=movie.get('title',0))
    m_g = widgets.HTML(value=movie.get('genre',0))
    m_y = widgets.HTML(value=movie.get('released',0))
    m_d = widgets.HTML(value=movie.get('director',0))
    m_c = widgets.HTML(value=movie.get('actors',0))
    m_p = widgets.HTML(value=movie.get('plot',0))
    m_r = widgets.HTML(value=movie.get('rated',0))
    m_a = widgets.HTML(value=movie.get('awards',0))
    m_x = widgets.HTML(value=movie.get('runtime',0))

    tab_nest = widgets.Tab()
    tab_nest.children = [m_t,m_g,m_y,m_d,m_c,m_p,m_r,m_a,m_x]
    tab_nest.set_title(0, 'Title')
    tab_nest.set_title(1, 'Genre')
    tab_nest.set_title(2, 'Released')
    tab_nest.set_title(3, 'Director')
    tab_nest.set_title(4, 'Actors')
    tab_nest.set_title(5, 'Plot')
    tab_nest.set_title(6, 'Rated')
    tab_nest.set_title(7, 'Awards')
    tab_nest.set_title(8, 'Runtime')  
    display(tab_nest)
    
class color:                                     # for editing text
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    UNDERLINE = '\033[4m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    END = '\033[0m'

The code above defines a function that takes the movie dictionary as an argument and uses the data to create a table which is displayed on screen whenever the user searches for a movie. The table was implemented by me as I wanted to know what GUI elements are available for Jupyter. As the widget is a table, I had to assign both tab headings and descriptions within each tab. Data from the movie dictionary also had to be converted into HTML before being used with the widget. Each individual tab within the table is referred to as a 'child'. I had to define the amount of 'children' wich the table will contain and nest the information in them along with their headings and heading positions. Naturally, the header position had to correspond with the position of the informtaion in the 'tab.children' list. 

The color class was used to add colour to the text displayed by the terminal. This was an easy way to add an extra feature without complicating the program too much. 

# REFERENCE LIST

- MuleSoft. 2019. What is a RESTful API. [Online]. Available: https://www.mulesoft.com/resources/api/restful-api [Accessed 10 May 2019].

- PyPi. 2018. OMDB pip installer. [Online]. Available: https://pypi.org/project/omdb/ [Accessed 6 May 2019].